In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from delta.tables import DeltaTable
from delta.pip_utils import configure_spark_with_delta_pip

# Create SparkSession
builder = (SparkSession.builder.appName("DeltaSession")
            .config("spark.jars.packages", )
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
          )

my_packages = [
    "io.delta:delta-core_2.12:2.3.0",
    "io.delta:delta-iceberg_2.12:2.3.0",
    "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.0.0"
]

spark = configure_spark_with_delta_pip(
            spark_session_builder=builder, 
            extra_packages=my_packages
        ).getOrCreate()

In [ ]:
# Define the schema for a DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
])

# Create a DataFrame using the schema
data = [(1, "Alice", 25, "New York"), 
        (2, "Bob", 30, "San Francisco"), 
        (3, "Charlie", 35, "Chicago")]

df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.toPandas()

In [ ]:
df.write.format("parquet").save("extract/07parquet")

In [ ]:
spark.read.format('parquet').load('extract/07parquet').toPandas()

## convertToDelta

In [ ]:
spark.sql("CONVERT TO DELTA parquet.`extract/07parquet`")

In [ ]:
DeltaTable.convertToDelta(spark, "parquet.`extract/07parquet`")

In [ ]:
df = spark.read.format("delta").load('extract/07parquet')

df.toPandas()

## [Convert a Delta table to a Parquet table](https://docs.delta.io/latest/delta-utility.html#convert-a-delta-table-to-a-parquet-table)

- run vacuum with retention of 0 hours to delete all data files that do not belong to the latest version of the table.
- Delete the _delta_log directory in the table directory.


## [Convert an Iceberg table to a Delta table](https://docs.delta.io/latest/delta-utility.html#convert-an-iceberg-table-to-a-delta-table)

In [ ]:
spark.sql(f"CONVERT TO DELTA local.db.iceberg")

In [ ]:
spark.sql(f"CONVERT TO DELTA iceberg.`extract/07iceberg/db/iceberg/`")

In [ ]:
path = "extract/07iceberg/db/iceberg"

spark.read.format('delta').load(path).toPandas()